# Azure ロールベースのアクセス制御 (Azure RBAC)

#### 前提条件
 - [PowerShell 7.X.X](https://docs.microsoft.com/ja-jp/powershell/azure/install-az-ps?view=azps-7.0.0)

## RBAC とは

ロールベースアクセス制御 : 

### できることは
- あるユーザーにサブスクリプション内の仮想マシンの管理を許可し、別のユーザーに仮想ネットワークの管理を許可
- DBA グループにサブスクリプション内の SQL データベースの管理を許可
- あるユーザーに、仮想マシン、Web サイト、サブネットなど、リソース グループ内のすべてのリソースの管理を許可
- あるアプリケーションに、リソース グループ内のすべてのリソースへのアクセスを許可

### 実施方法
ロールを割り当てる（以下、3要素）  

- セキュリティープリンシパル
- ロールの定義
- スコープ

### セキュリティープリンシパルとは
ユーザ、グループ、サービスプリンシパル、マネージドIDのこと

![](https://docs.microsoft.com/ja-jp/azure/role-based-access-control/media/shared/rbac-security-principal.png)  
出展 : [Azure公式 - セキュリティ プリンシパル -](https://docs.microsoft.com/ja-jp/azure/role-based-access-control/overview#security-principal)

### ロール定義
アクセス定義の設定情報（コレクションとよんでいる）通常は「ロール」と呼ばれる。
つまりは、どのような権限を与えるか という定義。  
[組み込み](https://docs.microsoft.com/ja-jp/azure/role-based-access-control/built-in-roles) と [カスタム](https://docs.microsoft.com/ja-jp/azure/role-based-access-control/custom-roles)がある。カスタムは有償プランが必要

![ロールの定義イメージ](https://docs.microsoft.com/ja-jp/azure/role-based-access-control/media/shared/rbac-role-definition.png)  
出展 : [Azure公式 - ロール定義 -](https://docs.microsoft.com/ja-jp/azure/role-based-access-control/overview#role-definition)  

ロール定義の仕組みの[詳細](https://docs.microsoft.com/ja-jp/azure/role-based-access-control/role-definitions)

### Scope（スコープ）

スコープは文字通り適用箇所になります。  
4つのレベルで指定ができ、下に行くほどスコープは小さくなります。
基本的に、役割を明確にすることで、余計な権限を与えないことがセキュリティー的には重要です。

![割り当てのイメージ](https://docs.microsoft.com/ja-jp/azure/role-based-access-control/media/shared/rbac-scope.png)  
出展 : [Azure公式 Score](https://docs.microsoft.com/ja-jp/azure/role-based-access-control/overview#scope)  

スコープについて詳しい話は[こちら](https://docs.microsoft.com/ja-jp/azure/role-based-access-control/scope-overview)参照


## PowerShell での操作

PowerShellにてロールの割り当て作業をする手順は以下4つの流れになる。  

1. 手順 1: アクセスが必要なユーザーを決定する
2. 手順 2:適切なロールを選択する
3. 手順 3:必要なスコープを特定する
4. 手順 4:ロールを割り当てる


[参考](https://docs.microsoft.com/ja-jp/azure/role-based-access-control/role-assignments-powershell)

### 手順 1: アクセスが必要なユーザーを決定する

### セキュリティープリンシパルの取得

#### ユーザ取得

[Get-AzADUser](https://docs.microsoft.com/ja-jp/powershell/module/az.resources/get-azaduser?view=azps-7.0.0)

In [ ]:
Get-AzADUser -StartsWith <userName>
(Get-AzADUser -DisplayName <userName>).id

#### グループ取得

[Get-AzADGroup](https://docs.microsoft.com/ja-jp/powershell/module/az.resources/get-azadgroup?view=azps-7.0.0)


In [ ]:
Get-AzADGroup -SearchString <groupName>
(Get-AzADGroup -DisplayName <groupName>).id

#### サービス プリンシパル

[Get-AzADServicePrincipal](https://docs.microsoft.com/ja-jp/powershell/module/az.resources/get-azadserviceprincipal?view=azps-7.0.0)

In [ ]:
Get-AzADServicePrincipal -SearchString <principalName>
(Get-AzADServicePrincipal -DisplayName <principalName>).id

#### 管理対象 ID

[Get-AzADServicePrincipal](https://docs.microsoft.com/ja-jp/powershell/module/az.resources/get-azadserviceprincipal?view=azps-7.0.0)

In [ ]:
Get-AzADServicePrincipal -SearchString <principalName>
(Get-AzADServicePrincipal -DisplayName <principalName>).id

### 手順 2:適切なロールを選択する


#### ロールの取得

必要最小限の特権を持つアクセス権を付与するのがベスト プラクティス  

取得したロールから必要なものを選択する。

In [ ]:
Get-AzRoleDefinition | FT Name, IsCustom, Id

In [ ]:
Get-AzRoleDefinition <roleName>

In [ ]:
# azuremlのロール取得
Get-AzRoleDefinition | Where-Object {$_.Name -match 'azureml'}
# azure blob

### 手順 3:必要なスコープを特定する

#### リソースのスコープ

4つのスコープから、最低限必要なスコープを選択する  

スコープの形式は以下
```
/subscriptions/<subscriptionId>/resourcegroups/<resourceGroupName>/providers/<providerName>/<resourceType>/<resourceSubType>/<resourceName>
```

それぞれの名称やIDを取得するためのコマンドは以下.

#### リソースグループのスコープ

リソースグループの名称を取得します

In [ ]:
Get-AzResourceGroup

#### サブスクリプションのスコープ

サブスクリプションIDを取得します。

In [ ]:
Get-AzSubscription

#### 管理グループ

管理グループ名が必要です

In [ ]:
Get-AzManagementGroup

### 手順 4:ロールを割り当てる

[New-AzRoleAssignment](https://docs.microsoft.com/ja-jp/powershell/module/az.resources/new-azroleassignment?view=azps-7.0.0)

#### リソースのスコープのサンプル

In [ ]:
New-AzRoleAssignment -ObjectId <objectId> `
-RoleDefinitionName <roleName> `
-Scope /subscriptions/<subscriptionId>/resourcegroups/<resourceGroupName>/providers/<providerName>/<resourceType>/<resourceSubType>/<resourceName>

In [ ]:
New-AzRoleAssignment -ObjectId <objectId> `
-RoleDefinitionId <roleId> `
-ResourceName <resourceName> `
-ResourceType <resourceType> `
-ResourceGroupName <resourceGroupName>